In [1]:
import numpy as np 
import pandas as pd 
import re
import time

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold,train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [2]:
def tokenize_data(data):
    global X, max_features
    max_features = 2000
    tokenizer = Tokenizer(num_words = max_features, split = ' ')
    tokenizer.fit_on_texts(data['preprocessedData'].values)
    X = tokenizer.texts_to_sequences(data['preprocessedData'].values)
    X = pad_sequences(X)

In [3]:
def setup_model():
    global model
    embed_dim = 128
    lstm_out = 196

    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
    #model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout = 0.2, recurrent_dropout = 0))
    model.add(Dense(3, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    #print(model.summary())    

In [4]:
def split_data():
    global train_index, test_index
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=2)
    iteration=0
    Y = pd.get_dummies(data['sentiment']).values
    
    for train_index, test_index in skf.split(data,data["sentiment"]):
        iteration=iteration+1
        start_time=time.time()
        setup_model()
        X_train, X_test, Y_train, Y_test=X[train_index], X[test_index], Y[train_index], Y[test_index]
        train_model(X_train, X_test, Y_train, Y_test, iteration, start_time)

        

In [5]:
def train_model(X_train, X_test, Y_train, Y_test, iteration, start_time):
    global batch_size, verbose 
    batch_size = 32
    epochs = 4
    verbose = 1 
    model.fit(X_train, Y_train, epochs = epochs, batch_size = batch_size, verbose = verbose)
    predict_model(X_train, X_test, Y_train, Y_test, iteration, start_time)

In [6]:
def predict_model(X_train, X_test, Y_train, Y_test, iteration, start_time):
    predicted_classes = []
    Y_test_reshape = []
    
    prediction = model.predict(X_test)
    for items in range(len(prediction)):
        predicted_classes.append(np.argmax(prediction[items]))
        Y_test_reshape.append(np.argmax(Y_test[items]))
        
    #print(predicted_classes)
    print('------------------')
    #print(Y_test_reshape)
    
    get_data(predicted_classes, Y_train, Y_test_reshape, iteration, start_time)

In [7]:
def get_data(predicted_classes, Y_train, Y_test_reshape, iteration, start_time):
    corporaType=""
    shortcut=""
    name=""
    totalTextUnits=0
    totalTestUnits=0
    totalTrainUnits=0
    accuracy=0
    f1_macro=0
    precision_macro=0
    recall_macro=0
    f1_micro=0
    precision_micro=0
    recall_micro=0
    matrix=[]
    f1_binary=0
    precision_binary=0
    recall_binary=0

    splitString=re.split("_|/",file)
    #print(splitString)
    corporaType=splitString[5]
    shortcut=splitString[6]
    name=splitString[7]

    totalTime=time.time()-start_time

    totalTextUnits=len(data.index)
    totalTestUnits=len(predicted_classes)
    totalTrainUnits=len(Y_train)

    accuracy=accuracy_score(Y_test_reshape, predicted_classes)
    
    f1_macro=f1_score(Y_test_reshape, predicted_classes, average="macro", labels=[1,0,2])
    precision_macro=precision_score(Y_test_reshape, predicted_classes, average="macro", labels=[1,0,2])
    recall_macro=recall_score(Y_test_reshape, predicted_classes, average="macro", labels=[1,0,2])

    f1_micro=f1_score(Y_test_reshape, predicted_classes, average="micro")
    precision_micro=precision_score(Y_test_reshape, predicted_classes, average="micro", labels=[1,0,2])
    recall_micro=recall_score(Y_test_reshape, predicted_classes, average="micro", labels=[1,0,2])

    matrix=confusion_matrix(Y_test_reshape, predicted_classes, labels=[1,0,2])

        
    matrixFlat=convertMatrix(matrix)
    target_names = [0,1,2]
    classificationReport=classification_report(Y_test_reshape, predicted_classes, target_names=target_names, output_dict=True)
    #print(matrixFlat)
    
    saveData(corporaType,shortcut,name,totalTime,totalTextUnits,totalTestUnits,totalTrainUnits,accuracy,f1_macro,precision_macro,recall_macro,
             f1_micro,precision_micro,recall_micro,matrix,f1_binary,precision_binary,recall_binary,predicted_classes, Y_test_reshape,start_time,iteration,matrixFlat,
             classificationReport)   

In [8]:
def convertMatrix(matrix):
    global flatMatrix
    array=[]
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            array.append(matrix[i][j])
    flatMatrix = np.array(array)
    return flatMatrix

In [9]:
def saveData(corporaType,shortcut,name,totalTime,totalTextUnits,totalTestUnits,totalTrainUnits,accuracy,f1_macro,precision_macro,recall_macro,f1_micro,precision_micro,recall_micro,matrix,f1_binary,precision_binary,recall_binary,y_test,test_set,startTime,iteration, matrixFlat,classificationReport):
    #print(matrixFlat)
    df_svm_data=pd.read_csv("RNNDataKFold.tsv", sep="\t")
    df_svm_data_full=pd.read_csv("RNNDataKFoldFull.tsv", sep="\t")

    #df=pd.DataFrame(columns=["Shortcut","Name","Type","Time","Total Length","Training Set","Test Set","Accuracy","Precision Macro",
    #          "Precision Micro","Precision Binary","Recall Macro","Recall Micro","Recall Binary","F1 Macro",
    #        "F1 Micro","F1 Binary","Matrix"])

    allData={"Iteration":iteration,"Shortcut":shortcut,"Name":name,"Type":corporaType,"Time":totalTime,"Total Length":totalTextUnits,"Training Set":totalTrainUnits,"Test Set":totalTestUnits,"Accuracy":accuracy,"Precision Macro":precision_macro,
             "Precision Micro":precision_micro,"Precision Binary":precision_binary,"Recall Macro":recall_macro,"Recall Micro":recall_micro,"Recall Binary":recall_binary,"F1 Macro":f1_macro,
            "F1 Micro":f1_micro,"F1 Binary":f1_binary,"Matrix":matrixFlat}

    allDataFull={"Iteration":iteration,"Shortcut":shortcut,"Name":name,"Type":corporaType,"Time":totalTime,"Total Length":totalTextUnits,"Training Set":totalTrainUnits,"Test Set":totalTestUnits,"Accuracy":accuracy,"Precision Macro":precision_macro,
             "Precision Micro":precision_micro,"Precision Binary":precision_binary,"Recall Macro":recall_macro,"Recall Micro":recall_micro,"Recall Binary":recall_binary,"F1 Macro":f1_macro,
            "F1 Micro":f1_micro,"F1 Binary":f1_binary,"Matrix":matrixFlat,"Train Set Full":y_test,"Test Set Full":test_set}

    
    reportDict=transformReport(classificationReport)
    allData.update(reportDict)
    allDataFull.update(reportDict)
    #print(allData)
    df_new_data=pd.DataFrame([allData])
    df_new_data_full=pd.DataFrame([allDataFull])

    finalData_svm=pd.concat([df_svm_data,df_new_data])
    finalData_svm_full=pd.concat([df_svm_data_full,df_new_data_full])
    
    finalData_svm=finalData_svm[["Iteration","Shortcut","Name","Type","Time","Total Length","Training Set","Test Set","Accuracy","Precision Macro",
              "Precision Micro","Precision Binary","Recall Macro","Recall Micro","Recall Binary","F1 Macro",
            "F1 Micro","F1 Binary","Matrix","0 precision","0 recall","0 f1-score",
            "0 support","1 precision","1 recall","1 f1-score","1 support","2 f1-score","2 support","2 precision","2 recall","accuracy accuracy",
            "macro avg precision","macro avg recall","macro avg f1-score","macro avg support","weighted avg precision","weighted avg recall",
            "weighted avg f1-score","weighted avg support"]]

    finalData_svm_full=finalData_svm_full[["Iteration","Shortcut","Name","Type","Time","Total Length","Training Set","Test Set","Accuracy","Precision Macro",
              "Precision Micro","Precision Binary","Recall Macro","Recall Micro","Recall Binary","F1 Macro",
            "F1 Micro","F1 Binary","Matrix","Train Set Full","Test Set Full","0 precision","0 recall","0 f1-score",
            "0 support","1 precision","1 recall","1 f1-score","1 support","2 f1-score","2 support","2 precision","2 recall","accuracy accuracy",
            "macro avg precision","macro avg recall","macro avg f1-score","macro avg support","weighted avg precision","weighted avg recall",
            "weighted avg f1-score","weighted avg support"]]
    
    finalData_svm.to_csv("RNNDataKFold.tsv", sep="\t",index=False)
    finalData_svm_full.to_csv("RNNDataKFoldFull.tsv", sep="\t",index=False)

In [10]:
def transformReport(classificationReport):
    #print(classificationReport)
    newDict={}
    for key in classificationReport.keys():
        mainName=str(key)
        if type(classificationReport[key]) != dict:
            name=mainName+" "+key
            newDict[name]=classificationReport[key]
            #print(key, "->",classificationReport[key],"-",)
            
        if type(classificationReport[key]) == dict:
            for k in classificationReport[key].keys():
                name=mainName+" "+k
                newDict[name]=classificationReport[key][k]
                #print(k,"->",classificationReport[key][k])
    #print("--------------")
    #print("DICT:",newDict)
    return newDict

files_binary=["../shared/Corpora/Preprocessed/Binary/LT01_gnd_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/LT02_speechLessing_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/LT03_historicplays_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/MI01_mlsa_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/MI02_germeval_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/MI03_corpusRauh_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/NA01_gersen_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/NA02_gerom_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/NA03_ompc_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/RE01_usage_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/RE03_critics_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/SM01_sb10k_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/SM02_potts_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/SM03_multiSe_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/SM04_gertwittersent_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/SM05_ironycorpus_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/SM06_celeb_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/RE02_scare_Preprocessed_binary_balanced.tsv",
                 "../shared/Corpora/Preprocessed/Binary/RE04_filmstarts_Preprocessed_binary.tsv",
                 "../shared/Corpora/Preprocessed/Binary/RE05_amazonreviews_Preprocessed_binary_balanced.tsv"]

In [11]:
files_ternary=["../../../Corpora/Preprocessed/Ternary/LT01_gnd_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/LT02_speechLessing_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/MI01_mlsa_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/MI02_germeval_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/MI03_corpusRauh_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/NA01_gersen_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/NA02_gerom_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/NA03_ompc_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/RE01_usage_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/RE03_critics_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/SM01_sb10k_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/SM02_potts_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/SM03_multiSe_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/SM04_gertwittersent_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/SM05_ironycorpus_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/SM06_celeb_Preprocessed_ternary.tsv",
                 "../../../Corpora/Preprocessed/Ternary/RE02_scare_Preprocessed_ternary_balanced.tsv",
                 "../../../Corpora/Preprocessed/Ternary/RE04_filmstarts_Preprocessed_ternary_all.tsv"
                 ]
def main():
    global data, file
    for file in files_ternary:
        print("Start for corpora: ", file)
        data=pd.read_csv(file, sep="\t")
        tokenize_data(data)
        split_data()
        print("Finish")

main()


Start for corpora:  ../../../Corpora/Preprocessed/Ternary/LT01_gnd_Preprocessed_ternary.tsv


2022-11-16 11:55:28.699278: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 11:55:29.121215: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22844 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:65:00.0, compute capability: 7.5


Epoch 1/4


2022-11-16 11:55:31.172330: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400


7/7 [==============================] - 2s 6ms/step - loss: 1.0822 - accuracy: 0.4604
Epoch 2/4
7/7 [==============================] - 0s 7ms/step - loss: 1.0335 - accuracy: 0.4604
Epoch 3/4
7/7 [==============================] - 0s 7ms/step - loss: 0.9940 - accuracy: 0.4604
Epoch 4/4
7/7 [==============================] - 0s 8ms/step - loss: 0.9267 - accuracy: 0.5891
------------------


...

Epoch 1/4
12/12 [==============================] - 1s 8ms/step - loss: 0.9797 - accuracy: 0.5163
Epoch 2/4
12/12 [==============================] - 0s 9ms/step - loss: 0.8268 - accuracy: 0.5679
Epoch 3/4
12/12 [==============================] - 0s 10ms/step - loss: 0.7689 - accuracy: 0.5734
Epoch 4/4
12/12 [==============================] - 0s 10ms/step - loss: 0.6763 - accuracy: 0.7228
------------------


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:13

Epoch 1/4
12/12 [==============================] - 1s 8ms/step - loss: 0.9397 - accuracy: 0.5245
Epoch 2/4
12/12 [==============================] - 0s 10ms/step - loss: 0.8277 - accuracy: 0.5353
Epoch 3/4
12/12 [==============================] - 0s 10ms/step - loss: 0.7830 - accuracy: 0.5679
Epoch 4/4
12/12 [==============================] - 0s 10ms/step - loss: 0.7247 - accuracy: 0.6495
------------------
Finish
Start for corpora:  ../../../Corpora/Preprocessed/Ternary/RE02_scare_Preprocessed_ternary_balanced.tsv


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:13

Epoch 1/4
1641/1641 [==============================] - 23s 14ms/step - loss: 0.8046 - accuracy: 0.6281
Epoch 2/4
1641/1641 [==============================] - 22s 14ms/step - loss: 0.7360 - accuracy: 0.6732
Epoch 3/4
1641/1641 [==============================] - 22s 14ms/step - loss: 0.7085 - accuracy: 0.6896
Epoch 4/4
1641/1641 [==============================] - 22s 14ms/step - loss: 0.6807 - accuracy: 0.7021
------------------
Epoch 1/4
1641/1641 [==============================] - 23s 14ms/step - loss: 0.8103 - accuracy: 0.6219
Epoch 2/4
1641/1641 [==============================] - 22s 14ms/step - loss: 0.7398 - accuracy: 0.6695
Epoch 3/4
1641/1641 [==============================] - 22s 14ms/step - loss: 0.7110 - accuracy: 0.6856
Epoch 4/4
1641/1641 [==============================] - 22s 14ms/step - loss: 0.6839 - accuracy: 0.7005
------------------
Epoch 1/4
1641/1641 [==============================] - 23s 14ms/step - loss: 0.8080 - accuracy: 0.6252
Epoch 2/4
1641/1641 [==============